# Creating a Module

In the following tutorial, we will be creating a `Module` object and demonstrating how to build a hierarchical SBOL model, including nested `ModuleDefinition`s and `MapsTo` objects.

`Module` objects have the following properties:
- `definition` : The definition property is a REQUIRED URI that refers to the `ModuleDefinition` for the `Module`.
- `mapsTo` : The mapsTos property is an OPTIONAL set of MapsTo objects that refer to and link ComponentInstance objects together within the heterarchy of Module, ModuleDefinition, ComponentInstance, and ComponentDefinition objects.
- `measures` : The measures property is OPTIONAL and MAY contain a set of `Measure` objects.

For more information on the `Module` class and its properties, check out page 42 of the SBOL 2.3.0 specifications which can be found at the following [link](https://sbolstandard.org/docs/SBOL2.3.0.pdf).

Import the module, create the document, and set the namespace.

In [1]:
import sbol2

doc = sbol2.Document()
sbol2.setHomespace("https://github.com/SynBioDex/SBOL-Notebooks")

## Define ComponentDefinitions

These `ComponentDefinition` objects represent the generic biological parts (genes, proteins, promoters) that will be instantiated as `FunctionalComponent`s within various modules. Adding them to the document allows them to be referenced.

In [2]:
# ComponentDefinition for TetR gene (DNA CDS)
tetR_gene_cd = sbol2.ComponentDefinition(uri='TetR_gene_definition', component_type=sbol2.BIOPAX_DNA)
tetR_gene_cd.addRole(sbol2.SO_CDS) # Coding Sequence
doc.addComponentDefinition(tetR_gene_cd)

# ComponentDefinition for TetR protein
tetR_protein_cd = sbol2.ComponentDefinition(uri='TetR_protein_definition', component_type=sbol2.BIOPAX_PROTEIN)
tetR_protein_cd.addRole(sbol2.BIOPAX_PROTEIN)
doc.addComponentDefinition(tetR_protein_cd)

# ComponentDefinition for LacI gene (DNA CDS)
lacI_gene_cd = sbol2.ComponentDefinition(uri='LacI_gene_definition', component_type=sbol2.BIOPAX_DNA)
lacI_gene_cd.addRole(sbol2.SO_CDS)
doc.addComponentDefinition(lacI_gene_cd)

# ComponentDefinition for LacI protein
lacI_protein_cd = sbol2.ComponentDefinition(uri='LacI_protein_definition', component_type=sbol2.BIOPAX_PROTEIN)
lacI_protein_cd.addRole(sbol2.BIOPAX_PROTEIN)
doc.addComponentDefinition(lacI_protein_cd)

# ComponentDefinition for pTet promoter
pTet_promoter_cd = sbol2.ComponentDefinition(uri='pTet_promoter_definition', component_type=sbol2.BIOPAX_DNA)
pTet_promoter_cd.addRole(sbol2.SO_PROMOTER)
doc.addComponentDefinition(pTet_promoter_cd)

# ComponentDefinition for pLac promoter
pLac_promoter_cd = sbol2.ComponentDefinition(uri='pLac_promoter_definition', component_type=sbol2.BIOPAX_DNA)
pLac_promoter_cd.addRole(sbol2.SO_PROMOTER)
doc.addComponentDefinition(pLac_promoter_cd)

## Create the `ModuleDefinition` for TetR Production

This module defines the process of TetR protein being produced from the TetR gene. It includes `FunctionalComponent`s for the gene and protein, and an `Interaction` describing the genetic production.

In [3]:
tetR_production_md = sbol2.ModuleDefinition('TetR_production_module')
doc.addModuleDefinition(tetR_production_md)

# Create FunctionalComponent for TetR gene (internal to this module)
tetR_gene_fc_internal = sbol2.FunctionalComponent('TetR_gene_fc_internal')
tetR_gene_fc_internal.definition = tetR_gene_cd.identity # Links to the common TetR gene CD
tetR_gene_fc_internal.access = sbol2.SBOL_ACCESS_PUBLIC
tetR_gene_fc_internal.direction = sbol2.SBOL_DIRECTION_NONE # Gene is a template
tetR_production_md.functionalComponents.add(tetR_gene_fc_internal)

# Create FunctionalComponent for TetR protein (internal to this module)
tetR_protein_fc_internal = sbol2.FunctionalComponent('TetR_protein_fc_internal')
tetR_protein_fc_internal.definition = tetR_protein_cd.identity # Links to the common TetR protein CD
tetR_protein_fc_internal.access = sbol2.SBOL_ACCESS_PUBLIC
tetR_protein_fc_internal.direction = sbol2.SBOL_DIRECTION_OUT # Protein is produced and exported
tetR_production_md.functionalComponents.add(tetR_protein_fc_internal)

# Create the Interaction for expressing TetR
tetR_expression_interaction = sbol2.Interaction('express_TetR_interaction')
tetR_expression_interaction.interaction_type = sbol2.SBO_GENETIC_PRODUCTION
tetR_production_md.interactions.add(tetR_expression_interaction)

# Add participants to the TetR expression interaction
# DNA template for production
dna_template_tetR_part = tetR_expression_interaction.participations.create('dna_template_tetR_part')
dna_template_tetR_part.roles = sbol2.SBOL_TEMPLATE
dna_template_tetR_part.participant = tetR_gene_fc_internal.identity # Participant is within this MD

# Produced protein
produced_protein_tetR_part = tetR_expression_interaction.participations.create('produced_protein_tetR_part')
produced_protein_tetR_part.roles = sbol2.SBO_PRODUCT
produced_protein_tetR_part.participant = tetR_protein_fc_internal.identity # Participant is within this MD

## Create the `ModuleDefinition` for LacI Production

Similar to the TetR production module, this defines the production of LacI protein from its gene.

In [4]:
lacI_production_md = sbol2.ModuleDefinition('LacI_production_module')
doc.addModuleDefinition(lacI_production_md)

# Create FunctionalComponent for LacI gene (internal to this module)
lacI_gene_fc_internal = sbol2.FunctionalComponent('LacI_gene_fc_internal')
lacI_gene_fc_internal.definition = lacI_gene_cd.identity # Links to the common LacI gene CD
lacI_gene_fc_internal.access = sbol2.SBOL_ACCESS_PUBLIC
lacI_gene_fc_internal.direction = sbol2.SBOL_DIRECTION_NONE
lacI_production_md.functionalComponents.add(lacI_gene_fc_internal)

# Create FunctionalComponent for LacI protein (internal to this module)
lacI_protein_fc_internal = sbol2.FunctionalComponent('LacI_protein_fc_internal')
lacI_protein_fc_internal.definition = lacI_protein_cd.identity # Links to the common LacI protein CD
lacI_protein_fc_internal.access = sbol2.SBOL_ACCESS_PUBLIC
lacI_protein_fc_internal.direction = sbol2.SBOL_DIRECTION_OUT
lacI_production_md.functionalComponents.add(lacI_protein_fc_internal)

# Create the Interaction for expressing LacI
lacI_expression_interaction = sbol2.Interaction('express_LacI_interaction')
lacI_expression_interaction.interaction_type = sbol2.SBO_GENETIC_PRODUCTION
lacI_production_md.interactions.add(lacI_expression_interaction)

# Add participants to the LacI expression interaction
dna_template_lacI_part = lacI_expression_interaction.participations.create('dna_template_lacI_part')
dna_template_lacI_part.roles = [sbol2.SBOL_TEMPLATE]
dna_template_lacI_part.participant = lacI_gene_fc_internal.identity

produced_protein_lacI_part = lacI_expression_interaction.participations.create('produced_protein_lacI_part')
produced_protein_lacI_part.roles = [sbol2.SBO_PRODUCT]
produced_protein_lacI_part.participant = lacI_protein_fc_internal.identity

## Create the Toggle Switch `ModuleDefinition`

This is the top-level module that orchestrates the repression between TetR and LacI. It instantiates the production modules as `Module` objects and defines the repression `Interaction`s at its own level. Crucially, it creates `FunctionalComponent`s that are *local* to this `ModuleDefinition` to participate in its `Interaction`s, and then uses `MapsTo` to link them to components in the sub-modules.

In [5]:
toggle_switch_md = sbol2.ModuleDefinition('Toggle_Switch_Module')
doc.addModuleDefinition(toggle_switch_md)

# Add instances of TetR and LacI production modules to the toggle switch
tetR_module_instance = sbol2.Module('TetR_production_instance')
tetR_module_instance.definition = tetR_production_md.identity
toggle_switch_md.modules.add(tetR_module_instance)

lacI_module_instance = sbol2.Module('LacI_production_instance')
lacI_module_instance.definition = lacI_production_md.identity
toggle_switch_md.modules.add(lacI_module_instance)

# Create FunctionalComponents within the Toggle Switch ModuleDefinition
# These represent the components that interact at the toggle switch level
tetR_protein_fc_parent = sbol2.FunctionalComponent('TetR_Protein_at_TS')
tetR_protein_fc_parent.definition = tetR_protein_cd.identity # Refers to the common protein CD
tetR_protein_fc_parent.access = sbol2.SBOL_ACCESS_PUBLIC
tetR_protein_fc_parent.direction = sbol2.SBOL_DIRECTION_NONE
toggle_switch_md.functionalComponents.add(tetR_protein_fc_parent)

lacI_protein_fc_parent = sbol2.FunctionalComponent('LacI_Protein_at_TS')
lacI_protein_fc_parent.definition = lacI_protein_cd.identity # Refers to the common protein CD
lacI_protein_fc_parent.access = sbol2.SBOL_ACCESS_PUBLIC
lacI_protein_fc_parent.direction = sbol2.SBOL_DIRECTION_NONE
toggle_switch_md.functionalComponents.add(lacI_protein_fc_parent)

# Promoters that are targets of repression in the toggle switch
pLac_promoter_fc_parent = sbol2.FunctionalComponent('pLac_promoter_at_TS')
pLac_promoter_fc_parent.definition = pLac_promoter_cd.identity # Refers to the common promoter CD
pLac_promoter_fc_parent.access = sbol2.SBOL_ACCESS_PUBLIC
pLac_promoter_fc_parent.direction = sbol2.SBOL_DIRECTION_NONE
toggle_switch_md.functionalComponents.add(pLac_promoter_fc_parent)

pTet_promoter_fc_parent = sbol2.FunctionalComponent('pTet_promoter_at_TS')
pTet_promoter_fc_parent.definition = pTet_promoter_cd.identity # Refers to the common promoter CD
pTet_promoter_fc_parent.access = sbol2.SBOL_ACCESS_PUBLIC
pTet_promoter_fc_parent.direction = sbol2.SBOL_DIRECTION_NONE
toggle_switch_md.functionalComponents.add(pTet_promoter_fc_parent)

# Create the repression interactions for the toggle switch
# Interaction: TetR represses LacI production
tetR_represses_lacI_interaction = sbol2.Interaction('TetR_represses_LacI_interaction')
tetR_represses_lacI_interaction.interaction_type = sbol2.SBO_INHIBITION
toggle_switch_md.interactions.add(tetR_represses_lacI_interaction)

# Participation: TetR as inhibitor
tetR_inhibitor_part = tetR_represses_lacI_interaction.participations.create('TetR_inhibitor_part')
tetR_inhibitor_part.roles = [sbol2.SBO_INHIBITOR]
tetR_inhibitor_part.participant = tetR_protein_fc_parent.identity # Correct: participant is in toggle_switch_md

# Participation: pLac_promoter as inhibited target
lacI_target_part = tetR_represses_lacI_interaction.participations.create('LacI_target_part')
lacI_target_part.roles = [sbol2.SBO_INHIBITED]
lacI_target_part.participant = pLac_promoter_fc_parent.identity # Correct: participant is in toggle_switch_md


# Interaction: LacI represses TetR production
lacI_represses_tetR_interaction = sbol2.Interaction('LacI_represses_TetR_interaction')
lacI_represses_tetR_interaction.interaction_type = sbol2.SBO_INHIBITION
toggle_switch_md.interactions.add(lacI_represses_tetR_interaction)

# Participation: LacI as inhibitor
lacI_inhibitor_part = lacI_represses_tetR_interaction.participations.create('LacI_inhibitor_part')
lacI_inhibitor_part.roles = [sbol2.SBO_INHIBITOR]
lacI_inhibitor_part.participant = lacI_protein_fc_parent.identity # Correct: participant is in toggle_switch_md

# Participation: pTet_promoter as inhibited target
tetR_target_part = lacI_represses_tetR_interaction.participations.create('TetR_target_part')
tetR_target_part.roles = [sbol2.SBO_INHIBITED]
tetR_target_part.participant = pTet_promoter_fc_parent.identity # Correct: participant is in toggle_switch_md

## Validate and Save the Document

Finally, validate the complete SBOL document and save it to an XML file. This step will now pass the validation, as the `FunctionalComponent`s referenced by `Participation` objects are correctly scoped within their containing `ModuleDefinition`.

In [6]:
report = doc.validate()
if (report == 'Valid.'):
    doc.write('module_example_fixed.xml')
    print("Validation successful. File saved as 'module_example_fixed.xml'")
else:
    print(report)

Validation successful. File saved as 'module_example_fixed.xml'
